<a href="https://colab.research.google.com/github/OussamaHaff/machine-learning-upskilling/blob/main/02-llms-from-scratch/04-token-embeddings-with-positional-encoding/token_embeddings_with_positisional_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic token IDs to embedding vectors

## Exploration

In [6]:
import torch

input_token_ids = torch.tensor([2, 3, 5, 1])

vocab_size = 6
output_embeddings_dim_size = 3

# Instantiate an embedding layer (with random seed 123 for reproducibility)
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size,
    embedding_dim=output_embeddings_dim_size)
print("Wights \n", embedding_layer.weight)
print("Shape\n", embedding_layer.weight.shape)

Wights 
 Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
Shape
 torch.Size([6, 3])


In [7]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [8]:
print(embedding_layer(input_token_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


## Observations

- Initial Embedding weights are random.
- The Embedding layer is essentially a lookup function to get wights of of token IDs.

## Mini positional embedding

In [9]:
max_length = 4
pos_embedding_layer = torch.nn.Embedding(
    num_embeddings=max_length, embedding_dim=output_embeddings_dim_size)

positions = torch.arange(max_length)

pos_embeddings = pos_embedding_layer(positions)

mini_embedding = embedding_layer(input_token_ids)

mini_input_embeddings = mini_embedding + pos_embeddings

print("Embeddings")
print(mini_embedding)
print("Positions")
print(positions)
print("Positional Embeddings")
print(pos_embeddings)
print("Target Embeddings")
print(mini_input_embeddings)

Embeddings
tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)
Positions
tensor([0, 1, 2, 3])
Positional Embeddings
tensor([[-0.6307,  1.2340,  0.3127],
        [ 0.6972, -0.9950, -1.1476],
        [-0.9178,  0.9045, -2.0975],
        [ 1.1558, -1.2157,  0.1295]], grad_fn=<EmbeddingBackward0>)
Target Embeddings
tensor([[ 0.6446,  1.0331,  0.1521],
        [ 0.2957, -0.0285, -2.2958],
        [-3.7578,  0.1197, -3.5071],
        [ 2.0735,  0.3653,  1.4306]], grad_fn=<AddBackward0>)


# Embedding with positional encoding

## From previous step

In [30]:
# Google Collab
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.4 MB/s eta 0:00:00


In [10]:
import torch
import tiktoken
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  """Contructs a PyTorch Dataset with """
  def __init__(self, text, tokeniser, sliding_window_max_length, stride):
    self.input_ids = []
    self.target_ids = []

    tokenised_text = tokeniser.encode(text)

    for i in range(0, len(tokenised_text) - sliding_window_max_length, stride):
      input_chunk = tokenised_text[i: i + sliding_window_max_length]
      target_chunk = tokenised_text[i + 1 : i + sliding_window_max_length + 1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, id):
    return self.input_ids[id], self.target_ids[id]


def create_dataloader_v1(text, batch_size=4, sliding_window_max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
  tokeniser = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(text, tokeniser, sliding_window_max_length, stride)
  dataloader = DataLoader(
      dataset,
      batch_size=batch_size,
      shuffle=shuffle,
      drop_last=drop_last,
      num_workers=num_workers
  )
  return dataloader


with open("data/the-verdict.txt", "r", encoding="utf-8") as dataset_file:
  raw_text = dataset_file.read()

In [11]:
vocab_size = 50257
output_dimension = 256

token_embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=output_dimension)
print(token_embedding_layer.weight.shape)

torch.Size([50257, 256])


In [12]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, sliding_window_max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token IDs:\n", inputs)
print("Input shape:\n", inputs.shape)


Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Input shape:
 torch.Size([8, 4])


In [13]:
tokens_embeddings = token_embedding_layer(inputs)

print(tokens_embeddings.shape)

torch.Size([8, 4, 256])


In [14]:
context_length = max_length
positional_embedding_layer = torch.nn.Embedding(
    num_embeddings=context_length, embedding_dim=output_dimension)
positional_embeddings = positional_embedding_layer(torch.arange(context_length))

print(positional_embeddings.shape)


torch.Size([4, 256])


In [15]:
input_embeddings = tokens_embeddings + positional_embeddings
print(input_embeddings)

tensor([[[-1.9733e+00, -1.1072e+00, -5.2032e-01,  ...,  8.7353e-01,
           4.6812e-01, -1.8006e+00],
         [ 9.4603e-01,  9.4199e-01,  1.4485e+00,  ...,  1.1931e+00,
           1.3748e+00,  3.0240e-01],
         [ 8.5075e-02, -2.6711e+00,  4.6235e-01,  ...,  7.9787e-01,
          -9.1028e-01, -1.8978e+00],
         [-7.2105e-01,  2.7123e+00,  9.3291e-01,  ...,  5.8263e-01,
          -3.6140e-01, -3.3350e-01]],

        [[-3.0873e+00, -7.2548e-01, -1.7576e+00,  ...,  6.8093e-02,
          -2.1905e+00,  1.1768e-01],
         [ 4.9539e-01, -3.5410e-01,  3.1399e-01,  ...,  5.4345e-02,
           4.2503e-01, -1.9022e+00],
         [ 1.1498e+00, -9.0849e-01,  7.0826e-01,  ...,  6.9908e-01,
           4.1368e-01, -1.4251e+00],
         [-2.9778e-01, -9.2440e-02, -1.9248e-01,  ...,  2.0688e+00,
          -4.6172e-01,  1.7991e+00]],

        [[-3.2814e+00, -1.1842e+00, -1.4793e+00,  ..., -3.6210e-01,
          -1.6815e+00, -1.0105e+00],
         [ 9.8176e-01,  1.2346e+00,  2.8820e+00,  .



The following illustration describes my understanding of the process of preparing data for training from tokenisation to positional embeddings.

![Understanding of tokenisation to positional embeddings](resources/tokenisation-to-positional-embeddings.png)

